In [ ]:
import numpy as np
import pandas as pd
import keras
from sklearn import metrics

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.model_selection import train_test_split,cross_validate
from keras.preprocessing.text import Tokenizer

import re
from sklearn.linear_model import LogisticRegression

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import WordNetLemmatizer  # lemmatizer from WordNet
from keras_preprocessing.sequence import pad_sequences

import gensim

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/My Drive/cyberbullying_tweets.csv')
df.head()

In [ ]:
df = df.rename(columns={"tweet_text": "tweet", "cyberbullying_type": "label"})

In [ ]:
print(df['tweet'].isnull().sum())
print(df['label'].isnull().sum())
print(df.shape)

In [ ]:
for tweet in df['tweet'][:5]:
    print(tweet+'\n'+'\n')

In [ ]:
print(df['label'].unique())

In [ ]:
def rename_label(label):
    if(label == 'not_cyberbullying'):
        return 0
    else:
        return 1

In [ ]:
df['label']=df['label'].apply(rename_label)
print(df['label'].unique())

In [ ]:
df['label'].value_counts()

In [ ]:
def cleanup(tweet):  
    
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub("[^a-zA-Z]"," ",tweet)
    word_tokens= tweet.lower().split()
    
    lemmatizer=WordNetLemmatizer()
    stop_words= set(stopwords.words("english"))     
    word_tokens= [lemmatizer.lemmatize(w) for w in word_tokens if not w in stop_words]
    
    return " ".join(word_tokens)

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
print(df.shape)

In [ ]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences=[]
sum=0

sumbeforecleaning=0
for tweet in df['tweet']:
    sumbeforecleaning +=len(tweet)
    cleaned_sents=cleanup(tweet.strip())
    sum+=len(cleaned_sents)
    sents=tokenizer.tokenize(cleaned_sents)
    for sent in sents:
        sentences.append(sent.split()) # can use word_tokenize also.

print(sumbeforecleaning)
print(sum)
print(len(sentences))  # total no of sentences

In [ ]:
for te in sentences[:5]:
    print(te,"\n")

In [ ]:
w2v_model=gensim.models.Word2Vec(sentences= sentences,size=300,window=10,min_count=1)
w2v_model.train(sentences,epochs=10,total_examples=len(sentences))

In [ ]:
w2v_model.wv.get_vector('bitch')

In [ ]:
vocab=w2v_model.wv.vocab
print("The total number of words are : ",len(vocab))

In [ ]:
w2v_model.wv.most_similar('hoe')

In [ ]:
w2v_model.wv.similarity('hoe','whore')

In [ ]:
vocab=list(vocab.keys())

In [ ]:
word_vec_dict={}
for word in vocab:
  word_vec_dict[word]=w2v_model.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict))
  

In [ ]:
df['tweet']=df['tweet'].apply(cleanup)

In [ ]:
maximum=-1
for i,rev in enumerate(df['tweet']):
    tokens=rev.split()
    if(len(tokens)>maximum):
        maximum=len(tokens)

In [ ]:
tok = Tokenizer()
tok.fit_on_texts(df['tweet'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['tweet'])

In [ ]:
max_len = 496
vocab_size = len(tok.word_index) + 1
embed_dim=300

In [ ]:
pad_rev_sampling= pad_sequences(x_text, maxlen=max_len, padding='post')
pad_rev_sampling.shape

In [ ]:
pad_rev= pad_sequences(encd_rev, maxlen=max_len, padding='post')
pad_rev.shape

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(pad_rev,df['label'],test_size=0.20,random_state=42, stratify = df['label'])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

logreg = LogisticRegression(max_iter = 5000)
logreg.fit(x_train, y_train)
y_pred_class = logreg.predict(x_test)
print((metrics.accuracy_score(y_test, y_pred_class)))

In [ ]:
print(precision_recall_fscore_support(y_test, y_pred_class, average = 'weighted'))